# Лабораторная работа номер 5

Выполнил: Стрельцов Илья Дмитриевич Б9122-01.03.02мкт

## Импорты

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision import models

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import kagglehub
from sklearn.metrics import roc_auc_score, roc_curve
import ipywidgets as widgets
from IPython.display import display, clear_output
import io

In [2]:
SEED = 5
IMG_SIZE = 150
BATCH_SIZE = 32
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

DATASET = 'sanikamal/horses-or-humans-dataset'
FILE_NAME = 'EXAMPLE.csv'

Using device: cpu


Преобразование теста

In [3]:
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

Преобразования валидации

In [4]:
val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

## Загрузка данных

In [5]:
dataset_path = kagglehub.dataset_download(DATASET)

100%|██████████| 307M/307M [06:12<00:00, 865kB/s]  

Extracting files...


'C:\\Users\\drumo\\.cache\\kagglehub\\datasets\\sanikamal\\horses-or-humans-dataset\\versions\\1'

In [10]:
# Определяем преобразования изображений
data_transform = transforms.Compose([
    transforms.Resize((150, 150)),  # Приводим все изображения к одному размеру
    transforms.ToTensor(),           # Преобразуем в тензоры PyTorch
    transforms.Normalize(            # Нормализуем значения пикселей
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Загружаем данные используя ImageFolder
train_dataset = datasets.ImageFolder(
    root=dataset_path+'/horse-or-human/train',
    transform=data_transform
)

validation_dataset = datasets.ImageFolder(
    root=dataset_path+'/horse-or-human/validation',
    transform=data_transform
)

# Создаем DataLoader для пакетной обработки
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=False)

# Проверяем что загрузилось
print(f"Классы: {train_dataset.classes}")
print(f"Соответствие классов индексам: {train_dataset.class_to_idx}")
print(f"Тренировочные примеры: {len(train_dataset)}")
print(f"Валидационные примеры: {len(validation_dataset)}")

Классы: ['horses', 'humans']
Соответствие классов индексам: {'horses': 0, 'humans': 1}
Тренировочные примеры: 1027
Валидационные примеры: 256
